In [1]:
import os
import json
import pandas as pd

import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from sklearn.metrics import classification_report
from transformers import Trainer, AutoModelForSequenceClassification


def get_text_encodings(model_name, texts1, texts2, max_length):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return tokenizer(texts1, texts2, 
                     truncation=True, 
                     padding="max_length", 
                     max_length=max_length)


class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) 
                for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


def get_dataset(model_name, texts1, texts2,
                max_length, labels):
    
    encodings = get_text_encodings(model_name, 
                                   texts1, texts2, 
                                   max_length)

    dataset = CustomDataset(encodings, labels)
    return dataset


def get_model_and_trainer(model_load_file):
    model = AutoModelForSequenceClassification.from_pretrained(model_load_file)
    trainer = Trainer(model=model)
    return model, trainer


def print_classification_report(y_test, y_pred):
    print(classification_report(y_test, y_pred, zero_division=0))


def sanity_check_av_ckpt_dir(ckpt_dir, test_set_fp=None, 
                             test_on_samples=False, samples_size=1000):
    ckpt_dir_parent = os.path.dirname(ckpt_dir)

    with open(os.path.join(ckpt_dir_parent, "args.json"), "r") as f:
        args = json.load(f)
    
    model_name = args["model_name"]
    max_length = args["max_length"]
    if test_set_fp is None:
        test_set_fp = os.path.join(args["data_dir"], "test.csv")
    
    test_set_fp = "../" + args["data_dir"] + "/test.csv"
    df = pd.read_csv(test_set_fp)

    if test_on_samples:
        df = df.sample(samples_size, random_state=42).reset_index(drop=True)

    labels = df["label"].tolist()
    dataset = get_dataset(model_name, df["text1"].tolist(), 
                          df["text2"].tolist(), max_length, labels)
    
    model, trainer = get_model_and_trainer(ckpt_dir)
    predictions = trainer.predict(dataset)
    y_pred = predictions.predictions.argmax(-1)

    model_name = model_name.split('/')[-1]
    prev_y_pred = df[f"{model_name}-prediction"]
    overlap = (y_pred == prev_y_pred).mean()
    print(f"Overlap: {overlap:.2f}")

    print_classification_report(y_pred, labels)
    

/home/jack/anaconda3/envs/PW/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-27 18:42:44.312600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745793764.324282 1916977 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745793764.327953 1916977 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745793764.338779 1916977 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.

In [2]:
ckpt_dir = "../AV_models/bert-base-uncased/blog_AV_datasets/checkpoint-125"
sanity_check_av_ckpt_dir(ckpt_dir, test_set_fp=None, 
                         test_on_samples=True, samples_size=1000)

Overlap: 1.00
              precision    recall  f1-score   support

           0       0.85      0.75      0.80       692
           1       0.56      0.71      0.63       308

    accuracy                           0.74      1000
   macro avg       0.71      0.73      0.71      1000
weighted avg       0.76      0.74      0.75      1000



In [3]:
ckpt_dir = "../AV_models/longformer-base-4096/enron_AV_datasets/checkpoint-250"
sanity_check_av_ckpt_dir(ckpt_dir, test_set_fp=None, 
                         test_on_samples=True, samples_size=1000)

Initializing global attention on CLS token...


Overlap: 1.00
              precision    recall  f1-score   support

           0       0.77      0.75      0.76       610
           1       0.62      0.64      0.63       390

    accuracy                           0.71      1000
   macro avg       0.69      0.70      0.69      1000
weighted avg       0.71      0.71      0.71      1000



In [4]:
ckpt_dir = "../AV_models/ModernBERT-base/blog_AV_datasets/checkpoint-250"
sanity_check_av_ckpt_dir(ckpt_dir, test_set_fp=None, 
                         test_on_samples=True, samples_size=1000)

Overlap: 1.00
              precision    recall  f1-score   support

           0       0.96      0.69      0.80       851
           1       0.32      0.83      0.46       149

    accuracy                           0.71      1000
   macro avg       0.64      0.76      0.63      1000
weighted avg       0.86      0.71      0.75      1000



In [ ]:
# infer_AV.py
import os
import json
import torch
import argparse
import pandas as pd
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from sklearn.metrics import classification_report
from torch.nn.functional import softmax
from transformers import Trainer, AutoModelForSequenceClassification


def get_args():
    parser = argparse.ArgumentParser(description="Inference script for AV model")
    parser.add_argument("--ckpt_dir", type=str, required=True, help="Directory containing the model checkpoint")
    parser.add_argument("--LLM_writing_dir_or_subdir", type=str, required=True, default="LLM_writing", 
                        help="LLM writing directory or subdirectory")
    return parser.parse_args()


def get_text_encodings(model_name, texts1, texts2, max_length):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return tokenizer(texts1, texts2, 
                     truncation=True, 
                     padding="max_length", 
                     max_length=max_length)


class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) 
                for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


def get_dataset(model_name, texts1, texts2,
                max_length, labels):
    
    encodings = get_text_encodings(model_name, 
                                   texts1, texts2, 
                                   max_length)

    dataset = CustomDataset(encodings, labels)
    return dataset


def get_model_and_trainer(ckpt_dir):
    model = AutoModelForSequenceClassification.from_pretrained(ckpt_dir)
    trainer = Trainer(model=model)
    return model, trainer


def print_classification_report(y_test, y_pred):
    print(classification_report(y_test, y_pred, zero_division=0))


def find_directories_with_file(root_dir, target_filename):
    matching_dirs = []

    for dirpath, _, filenames in os.walk(root_dir, topdown=False):
        if target_filename in filenames:
            matching_dirs.append(dirpath)

    return matching_dirs


def main():
    args = get_args()
    print(args)

    ckpt_dir_parent = os.path.dirname(args.ckpt_dir)
    with open(os.path.join(ckpt_dir_parent, "args.json"), "r") as f:
        args_dict = json.load(f)
    
    model, trainer = get_model_and_trainer(ckpt_dir)
    
    model_name = args_dict["model_name"]
    max_length = args_dict["max_length"]
    llm_dir = args.LLM_writing_dir_or_subdir

    dires_to_run = find_directories_with_file(llm_dir, "prompts.csv")
    for dire in dires_to_run:
        prompts_df = pd.read_csv(os.path.join(dire, "prompts.csv"))
        authors_texts = prompts_df["text"].tolist()
        
        for llm_fp in os.listdir(dire):
            if llm_fp.endswith(".csv") and llm_fp != "prompts.csv":
                llm_df = pd.read_csv(os.path.join(dire, llm_fp))

                if f"{model_name}-prediction" in llm_df.columns:
                    print(f"Already processed {llm_fp} in {dire}")
                    continue

                llm_texts = llm_df["writing"].tolist()

                if len(authors_texts) != len(llm_texts):
                    print(f"Length mismatch in {llm_fp} in {dire}")
                    continue
                    
                
                labels = [0] * len(llm_texts)
                dataset = get_dataset(model_name, authors_texts, 
                                      llm_texts, max_length, labels)
                predictions = trainer.predict(dataset)
                y_pred = predictions.predictions.argmax(-1)
                logits = predictions.predictions  # This contains the raw logits output
                # Convert logits to probabilities using softmax
                probabilities = softmax(torch.tensor(logits), dim=1).tolist()
                llm_df[f"{model_name}-prediction"]=y_pred
                llm_df[f"{model_name}-probabilities"] = [prob[1] for prob in probabilities]
                llm_df.to_csv(os.path.join(dire, llm_fp), index=False)
                print(f"Processed {llm_fp} in {dire}")
                

if __name__ == "__main__":
    main()